In [1]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 24.9 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opensearch-py


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests-aws4auth


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

In [ ]:
# Retrieve AWS credentials
credentials = boto3.Session().get_credentials()
#make sure you have set-up AWS cli in the environment you are executing this
# AWS authentication
region = "us-west-2"
service = "aoss"
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


In [ ]:
# Initialize OpenSearch client
client = OpenSearch(
    hosts=[{
        'host': 'your_opensearch_endpoint_url_from_aws_console',
        #remove the https from the prefix of url for the host
        'port': 443
    }],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

Deleting an index from a collection

In [ ]:
response = client.indices.delete(index="test", ignore=[400, 404]) 
#specify the index name or the list of index names to delete
response

Creating an index

In [1]:
# we need to create a schema for our index and follow the same layout while ingesting the documents
index_body={
  "settings": {
    "index": {
      "knn": True
    }
  },
  "mappings": {
    "properties": {
      "embedding": {
        "type": "knn_vector",
        "dimension": 1536  # This should match the dimension of your embeddings
      },
      "text": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 10000  # Set the maximum length for the keyword field
          }
        }
      },
      "metadata": {
        "properties": {
          "doc_name": {
            "type": "keyword",  # Use keyword type for exact matches
            "ignore_above": 1000  # Optional: Set a limit on the length of doc_name
          }
        }
      }
    }
  }
}


In [ ]:
index_name= "test"
client.indices.create(index=index_name, body=index_body)

Ingesting documents

In [ ]:
'''
    this cell uses some open source libraries that help us read and chunkify our text content.
    refer to the langchain documentation for more
'''
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from opensearchpy.helpers import bulk

types = ["type1", "type2", "type3"]
embeddings= your_embedding_model_class(config)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1600,
    separators=["\n\n", "."],
    chunk_overlap=400
)

for type in types:
    folder = f"C:/Users/karan_dhingra/Desktop/DirecTV/docs/{type}"
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

    for file in files:
        loader = PyPDFLoader(f"{folder}/{file}")
        documents = loader.load()
        docs = text_splitter.split_documents(documents)
        file_chunks = []
        
        # Assign metadata to each document
        for doc in docs:
            # Extract page_content and compute embedding
            page_content = doc.page_content
            embedding = embeddings.embed_query(page_content)
            index_document = {
                "_index": type,
                "_source": {
                    "text": page_content,
                    "embedding": embedding,
                    "metadata": {
                        "doc_name": file  # Add the file name as doc_name in metadata
                    }
                }
            }
            file_chunks.append(index_document)
        
        print(len(file_chunks))
        try:
            response = bulk(client, file_chunks)
        except Exception as e:
            print(f"Error occurred during bulk indexing: {e}")

# Check for any errors in the response and print them out.


Deleting a document

In [ ]:
''' 
    this function uses a term search query to get all the document ids matching a 
    criteria(doc_name being a certain value in this case) and use that list of ids to delete them
    from a certain index
'''
def delete_doc(doc_name):
    search_query = {
        "size": 100,  # Adjust the number of results as needed
        "query": {
            "term": {
                "metadata.doc_name": {
                    "value": doc_name
                }
            }
        }
    }
    # Perform the search
    response = client.search(
        index=types,  # Replace with the name of your index
        body=search_query
    )
    ids_to_del=[]
    for hit in response['hits']['hits']:
        ids_to_del.append(hit['_id'])

    if(len(response['hits']['hits'])):
         idx= response['hits']['hits'][0]['_index']
    for ids in ids_to_del:
        response = client.delete(index=idx, id=ids)
    return ids_to_del

res=delete_doc("test_doc.pdf")
res

Check whether an index exists

In [ ]:
client.indices.exists(index=index_name)